# <center> SQL Project: Chinook Database </center>
---

## Setup Environment
---

### Load libraries

In [1]:
import os
import glob
import sqlite3
import numpy as np
import pandas as pd
from contextlib import contextmanager

In [2]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

### Functions

In [3]:
@contextmanager
def connect_to_db(db_file: str = None):
    """Yield a sqlite3 connection to database file."""
    # Default database file is `../chinook_db/chinook.db`
    db_file = db_file or '../chinook_db/chinook.db'

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        yield conn
    finally:
        if conn:
            conn.close()

In [4]:
def get_dataframe_columns(**named_dfs):
    return {n: [*ndf.columns] for n,ndf in named_dfs.items()}

def display_dataframe_columns(df_columns: dict):
    for name, cols in df_columns.items():
        print(f'{name}_columns =\\ \n', cols)
        print()

In [5]:
def load_table(table_name: str, *, db_file: str = None):
    """Return a database table as a pandas.DataFrame."""
    with connect_to_db(db_file) as conn:
        return pd.read_sql(f'SELECT * FROM {table_name}', con=conn)    

In [6]:
def get_table_names(db_file: str = None) -> list[str]:
    """Return a list of all tables in a database."""
    with connect_to_db(db_file) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        return [table[0] for table in cursor.fetchall()]

chinook_tables = get_table_names()
print('- '+ '\n- '.join(chinook_tables))

- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- MediaType
- Playlist
- PlaylistTrack
- Track


### Classes

In [85]:
class Table:
    def __new__(cls):
        query = cls._get_query()
        with connect_to_db() as conn:
            return pd.read_sql(query, con=conn)
        
    @classmethod
    def _get_query(cls):
        query = (
        'SELECT'
        '   *'
        'FROM InvoiceLine;'
        )
        return query
    

table = Table()
table

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1
...,...,...,...,...,...
2235,2236,411,3136,0.99,1
2236,2237,411,3145,0.99,1
2237,2238,411,3154,0.99,1
2238,2239,411,3163,0.99,1


## Database Tables

---
- [Album](#album)
- [Artist](#artist)
- [Customer](#customer)
- [Employee](#employee)
- [Genre](#genre)
- [Invoice](#invoice)
- [InvoiceLine](#invoiceline)
- [MediaType](#mediatype)
- [Playlist](#playlist)
- [PlaylistTrack](#playlisttrack)
- [Track](#track)
---

### Albums

In [7]:
albums = load_table('Album')
albums.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   AlbumId   347 non-null    int64 
 1   Title     347 non-null    object
 2   ArtistId  347 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 8.3+ KB


In [8]:
albums

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


### Artists

In [9]:
artists = load_table('Artist')
artists.rename(columns=dict(Name='Artist'), inplace=True)
artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ArtistId  275 non-null    int64 
 1   Artist    275 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.4+ KB


In [10]:
artists

,ArtistId,Artist
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
...,...,...
270,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble


### Customers

In [11]:
customers = load_table('Customer')
customers.insert(1, 'Customer', customers['FirstName']+' '+customers['LastName'])
customers.rename(columns=dict(SupportRepId='EmployeeId', PostalCode='Zipcode'), inplace=True)
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerId  59 non-null     int64 
 1   Customer    59 non-null     object
 2   FirstName   59 non-null     object
 3   LastName    59 non-null     object
 4   Company     10 non-null     object
 5   Address     59 non-null     object
 6   City        59 non-null     object
 7   State       30 non-null     object
 8   Country     59 non-null     object
 9   Zipcode     55 non-null     object
 10  Phone       58 non-null     object
 11  Fax         12 non-null     object
 12  Email       59 non-null     object
 13  EmployeeId  59 non-null     int64 
dtypes: int64(2), object(12)
memory usage: 6.6+ KB


In [12]:
customers

,CustomerId,Customer,FirstName,LastName,Company,Address,City,State,Country,Zipcode,Phone,Fax,Email,EmployeeId
0,1,Luís Gonçalves,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie Köhler,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François Tremblay,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn Hansen,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František Wichterlová,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
5,6,Helena Holý,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
6,7,Astrid Gruber,Astrid,Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,None,astrid.gruber@apple.at,5
7,8,Daan Peeters,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,None,daan_peeters@apple.be,4
8,9,Kara Nielsen,Kara,Nielsen,None,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,+453 3331 9991,None,kara.nielsen@jubii.dk,4
9,10,Eduardo Martins,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4


### Employees

In [13]:
employees = load_table('Employee')
employees.insert(1, 'Employee', employees['FirstName']+' '+employees['LastName'])
employees.rename(columns=dict(PostalCode='Zipcode'), inplace=True)
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   EmployeeId  8 non-null      int64  
 1   Employee    8 non-null      object 
 2   LastName    8 non-null      object 
 3   FirstName   8 non-null      object 
 4   Title       8 non-null      object 
 5   ReportsTo   7 non-null      float64
 6   BirthDate   8 non-null      object 
 7   HireDate    8 non-null      object 
 8   Address     8 non-null      object 
 9   City        8 non-null      object 
 10  State       8 non-null      object 
 11  Country     8 non-null      object 
 12  Zipcode     8 non-null      object 
 13  Phone       8 non-null      object 
 14  Fax         8 non-null      object 
 15  Email       8 non-null      object 
dtypes: float64(1), int64(1), object(14)
memory usage: 1.1+ KB


In [14]:
employees

,EmployeeId,Employee,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,Zipcode,Phone,Fax,Email
0,1,Andrew Adams,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Nancy Edwards,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Jane Peacock,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Margaret Park,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Steve Johnson,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,6,Michael Mitchell,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
6,7,Robert King,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
7,8,Laura Callahan,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


### Genres

In [15]:
genres = load_table('Genre')
genres.rename(columns=dict(Name='Genre'), inplace=True)
genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   GenreId  25 non-null     int64 
 1   Genre    25 non-null     object
dtypes: int64(1), object(1)
memory usage: 532.0+ bytes


In [16]:
genres

,GenreId,Genre
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll
5,6,Blues
6,7,Latin
7,8,Reggae
8,9,Pop
9,10,Soundtrack


### Invoices

In [17]:
invoices = load_table('Invoice')
invoices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412 entries, 0 to 411
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   InvoiceId          412 non-null    int64  
 1   CustomerId         412 non-null    int64  
 2   InvoiceDate        412 non-null    object 
 3   BillingAddress     412 non-null    object 
 4   BillingCity        412 non-null    object 
 5   BillingState       210 non-null    object 
 6   BillingCountry     412 non-null    object 
 7   BillingPostalCode  384 non-null    object 
 8   Total              412 non-null    float64
dtypes: float64(1), int64(2), object(6)
memory usage: 29.1+ KB


In [18]:
invoices

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...
407,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


### InvoiceLines

In [19]:
invoice_lines = load_table('InvoiceLine')
invoice_lines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   InvoiceLineId  2240 non-null   int64  
 1   InvoiceId      2240 non-null   int64  
 2   TrackId        2240 non-null   int64  
 3   UnitPrice      2240 non-null   float64
 4   Quantity       2240 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 87.6 KB


In [20]:
invoice_lines

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1
...,...,...,...,...,...
2235,2236,411,3136,0.99,1
2236,2237,411,3145,0.99,1
2237,2238,411,3154,0.99,1
2238,2239,411,3163,0.99,1


### MediaTypes

In [21]:
mediatypes = load_table('MediaType')
mediatypes.rename(columns=dict(Name='MediaType'), inplace=True)
mediatypes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MediaTypeId  5 non-null      int64 
 1   MediaType    5 non-null      object
dtypes: int64(1), object(1)
memory usage: 212.0+ bytes


In [22]:
mediatypes

,MediaTypeId,MediaType
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


### Playlists

In [23]:
playlists = load_table('Playlist')
playlists.rename(columns=dict(Name='Playlist'), inplace=True)
playlists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PlaylistId  18 non-null     int64 
 1   Playlist    18 non-null     object
dtypes: int64(1), object(1)
memory usage: 420.0+ bytes


In [24]:
playlists

,PlaylistId,Playlist
0,1,Music
1,2,Movies
2,3,TV Shows
3,4,Audiobooks
4,5,90’s Music
5,6,Audiobooks
6,7,Movies
7,8,Music
8,9,Music Videos
9,10,TV Shows


### PlaylistTracks

In [25]:
playlist_tracks = load_table('PlaylistTrack')
playlist_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8715 entries, 0 to 8714
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   PlaylistId  8715 non-null   int64
 1   TrackId     8715 non-null   int64
dtypes: int64(2)
memory usage: 136.3 KB


In [26]:
playlist_tracks

,PlaylistId,TrackId
0,1,3402
1,1,3389
2,1,3390
3,1,3391
4,1,3392
...,...,...
8710,17,2094
8711,17,2095
8712,17,2096
8713,17,3290


### Tracks

In [27]:
tracks = load_table('Track')
tracks.rename(columns=dict(Name='Track'), inplace=True)
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3503 entries, 0 to 3502
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TrackId       3503 non-null   int64  
 1   Track         3503 non-null   object 
 2   AlbumId       3503 non-null   int64  
 3   MediaTypeId   3503 non-null   int64  
 4   GenreId       3503 non-null   int64  
 5   Composer      2525 non-null   object 
 6   Milliseconds  3503 non-null   int64  
 7   Bytes         3503 non-null   int64  
 8   UnitPrice     3503 non-null   float64
dtypes: float64(1), int64(6), object(2)
memory usage: 246.4+ KB


In [28]:
tracks

,TrackId,Track,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


## Exploratory Data Analysis EDA
---

## Data Modeling
---

### Fact Tables for Measurement
- [Invoice Features](#invoice-features)
- [InvoiceLine Features](#invoiceline-features)
- [PlaylistTrack Features](#playlisttrack-features)

#### Invoice Features

- `CustomerId` $\in$ __Customer__ $\to$ `EmployeeId` $\in$ __Employee__
- `CustomerId` $\to$ __Customer__ 

In [29]:
invoices

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...
407,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


In [32]:
frames = get_dataframe_columns(customers=customers,
                               employees=employees,
                               invoices=invoices)

display_dataframe_columns(frames)

customers_columns =\ 
 ['CustomerId', 'Customer', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'Zipcode', 'Phone', 'Fax', 'Email', 'EmployeeId']

employees_columns =\ 
 ['EmployeeId', 'Employee', 'LastName', 'FirstName', 'Title', 'ReportsTo', 'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country', 'Zipcode', 'Phone', 'Fax', 'Email']

invoices_columns =\ 
 ['InvoiceId', 'CustomerId', 'InvoiceDate', 'BillingAddress', 'BillingCity', 'BillingState', 'BillingCountry', 'BillingPostalCode', 'Total']



In [ ]:
frames = dict(customers=customers, employees=employees, invoices=invoices)

for name, frame in frames.items():
    print(f'{name}_columns =', [*frame.columns])
    print()

# Set the customer columns to be extracted
customers_columns = ['CustomerId', 'Customer', 'City', 'Country', 'Zipcode', 'Phone', 'Email', 'EmployeeId']

# Set the employee columns to be extracted
employees_columns = ['EmployeeId', 'Employee', 'Title', 'ReportsTo', 'City', 'Country', 'Phone', 'Email']

# Set the invoice columns to be extracted
invoices_columns = ['InvoiceId', 'CustomerId', 'InvoiceDate', 'Total']


invoices.merge(
    customers[customers_columns].merge(employees[employees_columns], on='EmployeeId', suffixes=('Customer','Employee'))
)

#### InvoiceLine Features

In [43]:
def get_union_columns(left: pd.DataFrame, right: pd.DataFrame):
    return set(*left.columns).add(*right.columns)

In [44]:
from functools import reduce

In [63]:
# get_union_columns(customers, empl)

# reduce(get_union_columns, [invoice_lines, customers, employees])

set(invoice_lines.columns).intersection(set(customers.columns))  # .intersection(employees.columns)

def get_columns_intersection(left, right):
    return {l for l in left.columns if 'Id' in l and l in right.columns}


# get_columns_intersection(employees, customers)

In [71]:
set.intersection(set(employees.columns), set(customers.columns))

reduce(set.intersection, map(lambda x: set(x.columns), [employees, customers,]))

{'Address',
 'City',
 'Country',
 'Email',
 'EmployeeId',
 'Fax',
 'FirstName',
 'LastName',
 'Phone',
 'State',
 'Zipcode'}

- __InvoiceLine:__ $\left( \rm{InvoiceId} \in \rm{Invoice} \subset \right)$

In [76]:
frames = get_dataframe_columns(invoice_lines=invoice_lines,
                               invoices=invoices,
                               customers=customers,
                               employees=employees,
                               )

display_dataframe_columns(frames)

print()

frames = get_dataframe_columns(invoice_lines=invoice_lines,
                               tracks=tracks,
                               albums=albums,
                               mediatypes=mediatypes,
                               genres=genres,
                               artists=artists,
                               )

display_dataframe_columns(frames)

get_columns_intersection(employees, customers)

invoice_lines_columns =\ 
 ['InvoiceLineId', 'InvoiceId', 'TrackId', 'UnitPrice', 'Quantity']

invoices_columns =\ 
 ['InvoiceId', 'CustomerId', 'InvoiceDate', 'BillingAddress', 'BillingCity', 'BillingState', 'BillingCountry', 'BillingPostalCode', 'Total']

customers_columns =\ 
 ['CustomerId', 'Customer', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'Zipcode', 'Phone', 'Fax', 'Email', 'EmployeeId']

employees_columns =\ 
 ['EmployeeId', 'Employee', 'LastName', 'FirstName', 'Title', 'ReportsTo', 'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country', 'Zipcode', 'Phone', 'Fax', 'Email']


invoice_lines_columns =\ 
 ['InvoiceLineId', 'InvoiceId', 'TrackId', 'UnitPrice', 'Quantity']

tracks_columns =\ 
 ['TrackId', 'Track', 'AlbumId', 'MediaTypeId', 'GenreId', 'Composer', 'Milliseconds', 'Bytes', 'UnitPrice']

albums_columns =\ 
 ['AlbumId', 'Title', 'ArtistId']

mediatypes_columns =\ 
 ['MediaTypeId', 'MediaType']

genres_columns =\ 
 ['GenreId', 'Ge

{'EmployeeId'}

In [78]:
class InvoiceLineCustomerFacts:
    def __new__(cls):
        cls.invoice_lines = load_table('InvoiceLine')
        cls.invoices = load_table('Invoice')
        cls.customers = load_table('Customer')
        return invoice_lines.merge(invoices).merge(customers)
        

InvoiceLineCustomerFacts()

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,...,Company,Address,City,State,Country,Zipcode,Phone,Fax,Email,EmployeeId
0,1,1,2,0.99,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,...,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
1,2,1,4,0.99,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,...,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,2,6,0.99,1,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,...,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
3,4,2,8,0.99,1,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,...,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,2,10,0.99,1,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,...,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,2236,411,3136,0.99,1,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,...,None,Porthaninkatu 9,Helsinki,None,Finland,00530,+358 09 870 2000,None,terhi.hamalainen@apple.fi,3
2236,2237,411,3145,0.99,1,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,...,None,Porthaninkatu 9,Helsinki,None,Finland,00530,+358 09 870 2000,None,terhi.hamalainen@apple.fi,3
2237,2238,411,3154,0.99,1,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,...,None,Porthaninkatu 9,Helsinki,None,Finland,00530,+358 09 870 2000,None,terhi.hamalainen@apple.fi,3
2238,2239,411,3163,0.99,1,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,...,None,Porthaninkatu 9,Helsinki,None,Finland,00530,+358 09 870 2000,None,terhi.hamalainen@apple.fi,3


In [ ]:
def get_invoice_line_customer_facts(invoice_lines: pd.DataFrame, customer):

In [ ]:
def merge_dataframes(left: pd.DataFrame, right: pd.DataFrame) -> pd.DataFrame:
    left.merge(right, on=on, )

In [1]:
def get_artist_features_of_invoice():
    pass

In [ ]:
pd.D

In [ ]:
frames = dict(customers=customers, albums=albums, artists=artists, )
invoice_lines.merge(invoices.merge(tracks, suffixes=('Customer','')))

#### PlaylistTrack Features

### Dimension Tables for Descriptive Context
---
- [Album](#album-dimensions)
- [Artist](#artist-dimensions)
- [Customer](#customer-dimensions)
- [Employee](#employee-dimensions)
- [Genre](#genre-dimensions)
- [MediaType](#mediatype-dimensions)
- etc.
---

#### Album Dimensions

#### Artist Dimensions

#### Customer Dimensions

#### Employee Dimensions

#### Genre Dimensions

#### MediaType Dimensions

---
---
---

In [41]:
def get_null_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Return a pandas dataframe with columns of NaN counts & percents."""
    null_data = [df.isna().sum(), 100*(df.isna().sum()/len(df))]
    return pd.concat(null_data, keys=['null_count','null_percent'], axis=1).round(1)

In [ ]:

get_null_dataframe(customers)

In [ ]:

get_null_dataframe(employees)

In [ ]:
# Explore Invoices Table
invoices

# Set columns for output dataframe
invoice_columns = [
    'InvoiceId', 'CustomerId', 'InvoiceDate', 'Total'
]

# Customer Columns:
#     'CustomerId', 'Customer', 'FirstName', 'LastName', 'Company',
#     'Address', 'City', 'State', 'Country', 'Zipcode', 'Phone',
#     'Fax', 'Email', 'EmployeeId'
customer_columns = [
    'CustomerId', 'Customer', 'City', 'Country',
    'Phone', 'Email', 'EmployeeId',
    ]

# Employee Columns:
#     'EmployeeId', 'Employee', 'LastName', 'FirstName', 'Title',
#     'ReportsTo', 'BirthDate', 'HireDate', 'Address', 'City',
#     'State', 'Country', 'Zipcode', 'Phone', 'Fax', 'Email'
employee_columns = [
    'EmployeeId', 'Employee', 'Title',
    'BirthDate', 'HireDate',
    'Phone', 'Fax', 'Email',
]


df = invoices[invoice_columns]\
    .merge(customers[customer_columns], on='CustomerId',)\
    .merge(employees[employee_columns], on='EmployeeId',
           suffixes=('_customer','_employee'))

df.groupby()

In [ ]:
? pd.DataFrame.merge

In [ ]:
# Invoices
df = invoices.merge(customers).merge(employees, left_on='SupportRepId')

# Set columns for union dataframe
invoice_columns = ['InvoiceId', 'CustomerId', 'InvoiceDate', 'Total']
customer_columns = ['CustomerId', 'Customer', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
employee_columns = ['EmployeeId', 'Employee', 'LastName', 'FirstName', 'Title', 'ReportsTo', 'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email']



---

---
---
---